# Regressão Logística I
## Tarefa I

Neste exercício vamos trabalhar com a base "Heart Disease Data Set" disponível no site do [UCI](https://archive.ics.uci.edu/ml/datasets/heart+disease).

Temos 303 indivíduos e vamos utilizar 13 variáveis disponíveis para prever a presença de doença cardíaca na variável ```num```(a 14$^a$ variável).

A descrição das variáveis está recortada abaixo:
- age: idade do paciente em anos
- sex: sexo (1 = male; 0 = female)  
- cp: tipo de dor no peito
  - 1: angina típica
  - 2: angina atípica
  - 3: dor não-angina
  - 4: assintomático
- trestbps: pressão sanguínea em repouso (em mm Hg na admissão ao hospital
- chol: colesterol sérico em mg/dl
- fbs: (açúcar no sangue em jejum > 120 mg/dl) (1 = True; 0 = False)
- restecg: resultados eletrocardiográficos em repouso
  - 0: normal
  - 1: tendo anormalidade da onda ST-T (Inversões de onda T e / ou ST com elevação ou depressão de > 0.05 mV)
  - 2: mostrando hipertrofia ventricular esquerda provável ou definitiva pelos critérios de Estes
- thalach: frequência cardíaca máxima alcançada
- exang: angina induzida por exercício(1 = sim; 0 = não)
- oldpeak = Depressão de ST induzida por exercício em relação ao repouso
- slope: Depressão de ST induzida por exercício em relação ao repouso
  - 1: inclinação ascendente
  - 2: estável
  - 3: inclinação descendente
- ca: número de vasos principais (0-3) coloridos por fluorosopia
- thal: 3 = normal; 6 = defeito corrigido; 7 = defeito reversível
- num: diagnóstico de doença cardíaga (status de doença angiográfica)

In [98]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

import statsmodels.formula.api as smf

In [99]:
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data'

df = pd.read_csv(url,
                 names=['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg',
                        'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'num'])
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,num
0,63.0,1.0,1.0,145.0,233.0,1.0,2.0,150.0,0.0,2.3,3.0,0.0,6.0,0
1,67.0,1.0,4.0,160.0,286.0,0.0,2.0,108.0,1.0,1.5,2.0,3.0,3.0,2
2,67.0,1.0,4.0,120.0,229.0,0.0,2.0,129.0,1.0,2.6,2.0,2.0,7.0,1
3,37.0,1.0,3.0,130.0,250.0,0.0,0.0,187.0,0.0,3.5,3.0,0.0,3.0,0
4,41.0,0.0,2.0,130.0,204.0,0.0,2.0,172.0,0.0,1.4,1.0,0.0,3.0,0


In [100]:
df['flag_doente'] = (df['num'] != 0).astype('int64')
df.head(4)

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,num,flag_doente
0,63.0,1.0,1.0,145.0,233.0,1.0,2.0,150.0,0.0,2.3,3.0,0.0,6.0,0,0
1,67.0,1.0,4.0,160.0,286.0,0.0,2.0,108.0,1.0,1.5,2.0,3.0,3.0,2,1
2,67.0,1.0,4.0,120.0,229.0,0.0,2.0,129.0,1.0,2.6,2.0,2.0,7.0,1,1
3,37.0,1.0,3.0,130.0,250.0,0.0,0.0,187.0,0.0,3.5,3.0,0.0,3.0,0,0


1. Monte uma bivariada para a variável ```sex``` com as seguintes estatísticas:

- Número de *saudáveis*
- Número de *doentes*
- Total
- Quantidade média de doentes
- *Odds*
- *Odds ratio* (Use o total como referência)
- Logito (LOG(*Odds*))
- O LOG do *Odds ratio* (com o total como referência) é chamado de *Weight of Evidence* (WOE). Calcule-o.

In [101]:
biv = df.groupby(['sex', 'flag_doente']).size().reset_index(name='count')

In [102]:
biv['saudavel'] = biv.apply(lambda row: row['count'] if row['flag_doente'] == 0 else 0, axis=1)
biv['doente'] = biv.apply(lambda row: row['count'] if row['flag_doente'] == 1 else 0, axis=1)

In [103]:
biv = biv.groupby("sex").sum().reset_index().rename(columns={"count": "total"})

In [104]:
biv["media doentes"] = biv["total"] / biv["doente"]

In [105]:
biv["odds"] = (biv["doente"] / biv["total"]) / (biv["saudavel"] / biv["total"])

In [106]:
biv["odds ratio"] = biv["odds"] / biv["total"]

In [107]:
biv["logito"] = np.log(biv["odds"])

In [108]:
biv["woe"] = np.log(biv["odds ratio"])

In [109]:
biv.drop(columns=["flag_doente"])

,sex,total,saudavel,doente,media doentes,odds,odds ratio,logito,woe
0,0.0,97,72.0,25.0,3.880000,0.347222,0.003580,-1.05779,-5.632501
1,1.0,206,92.0,114.0,1.807018,1.239130,0.006015,0.21441,-5.113466


2. Monte uma bivariada para a variável ```age``` com as seguintes estatísticas:
- Categorize a variável em 5 grupos
- Calcule as mesmas quantidades calculadas no execício anterior
- Calcule também o valor médio da variável ```age```

In [110]:
bins = [0, 40, 50, 60, 70, 80]
categorias = ["0-40", "41-50", "51-60", "61-70", "71-80"]

In [111]:
df["age-group"] = pd.cut(df["age"], bins=bins, labels=categorias, right=False)

In [112]:
df["age-group"].value_counts()

age-group
51-60    125
61-70     81
41-50     72
0-40      15
71-80     10
Name: count, dtype: int64

In [113]:
biv = df.groupby(['age-group', 'flag_doente']).size().reset_index(name='count')

In [114]:
biv['saudavel'] = biv.apply(lambda row: row['count'] if row['flag_doente'] == 0 else 0, axis=1)
biv['doente'] = biv.apply(lambda row: row['count'] if row['flag_doente'] == 1 else 0, axis=1)

In [115]:
biv = biv.groupby("age-group").sum().reset_index().rename(columns={"count": "total"})

In [116]:
biv["media doentes"] = biv["total"] / biv["doente"]

In [117]:
biv["odds"] = (biv["doente"] / biv["total"]) / (biv["saudavel"] / biv["total"])

In [118]:
biv["odds ratio"] = biv["odds"] / biv["total"]

In [120]:
biv["logito"] = np.log(biv["odds"])

In [121]:
biv["woe"] = np.log(biv["odds ratio"])

In [138]:
biv.sort_values(by="total", inplace=True)

In [139]:
biv

,age-group,total,saudavel,doente,media doentes,odds,odds ratio,logito,woe
4,71-80,10,6,4,2.500000,0.666667,0.066667,-0.405465,-2.708050
0,0-40,15,11,4,3.750000,0.363636,0.024242,-1.011601,-3.719651
1,41-50,72,50,22,3.272727,0.440000,0.006111,-0.820981,-5.097647
3,61-70,81,32,49,1.653061,1.531250,0.018904,0.426084,-3.968365
2,51-60,125,65,60,2.083333,0.923077,0.007385,-0.080043,-4.908356


3. Visualize a informação da bivariada construída acima através de um gráfico da sua escolha.

In [163]:
import plotly.express as px
import plotly.graph_objects as go

In [164]:
fig = go.Figure()

In [165]:
for column in biv.columns:
  if column not in ["age-group", "total"]:
    fig.add_trace(
        go.Scatter(
          x=biv["total"],
          y=biv[column],
          name=column
      )
    )

    fig.update_layout(xaxis_title="Total")

In [166]:
fig.show()